# __Datenbereinigung Teil 4: Bereinigung der Anforderungsdaten__

In diesem Schritt werden die Daten bereinigt, welche für das Wörterbuch mit den häufigst genannten Jobtiteln benötigt werden.

In [17]:
# Import der benötigten Libraries
from time import time
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from bs4 import BeautifulSoup
from HanTa import HanoverTagger as ht
import string
from nltk.tokenize.treebank import TreebankWordDetokenizer
import os

# Working-Directory
os.chdir('C:/Users/Sven Konermann/Documents/Master/2_Semester/aF_E_Faelle/Unterlagen/00_all')

In [18]:
# Einlesen der Daten über ein csv File
data = pd.read_csv("Inserate_Anforderungsanalyse_AF_E.csv", sep=";")
print(data.shape)
data.info()

(25100, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25100 entries, 0 to 25099
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0.1           25100 non-null  int64  
 1   Unnamed: 0             25100 non-null  int64  
 2   ad_id                  25100 non-null  object 
 3   active_start_on        25100 non-null  object 
 4   active_end_on          25100 non-null  object 
 5   title                  25100 non-null  object 
 6   text                   25100 non-null  object 
 7   branchen_name          24926 non-null  object 
 8   customer_branche_name  25099 non-null  object 
 9   lvl1_region_name       25065 non-null  object 
 10  region_name            25065 non-null  object 
 11  lvl1_jobcategory_name  25093 non-null  object 
 12  jobcategory_name       25093 non-null  object 
 13  duration               25100 non-null  float64
 14  year                   25100 non-null  int

In [22]:
# Listenelemente in einer neuen Spalte abspeichern
start = time()

column = []
for i in data['title']:
    title= i
    #soup = BeautifulSoup(i, 'lxml')
    #tags = soup.find_all('li')
    #print(tags)
    for t in tags:
        title = title + ' ' + t.title 
    column.append(title)

data['Title_Anforderungen'] = column

print(f'Benötigte Zeit: {time() - start} Sekunden')
data.head(5)

Benötigte Zeit: 0.024413108825683594 Sekunden


,Unnamed: 0.1,Unnamed: 0,ad_id,active_start_on,active_end_on,title,text,branchen_name,customer_branche_name,lvl1_region_name,region_name,lvl1_jobcategory_name,jobcategory_name,duration,year,Title_Anforderungen
0,7,7,jobsse-5879310,2014-06-19,2014-07-14,Projektleiter SAP Inkasso / Exkasso (FS-CD) (w/m),"<div id=""jobTplContainer"" style=""overflow: hid...",Versicherungen,Versicherungen,Region Ostschweiz / GR / FL,St. Gallen / Appenzell,Informatik/Telekommunikation,ERP/SAP/CRM,25.0,2014,Projektleiter SAP Inkasso / Exkasso (FS-CD) (w/m)
1,19,19,jobsse-5707516,2014-03-19,2014-04-20,IT Projektleiter/-in,<p>In dieser Position sind Sie verantwortlich ...,Banken/ Finanzinstitute,Banken/ Finanzinstitute,Region ZÇ¬rich / Schaffhausen,Stadt ZÇ¬rich / ZÇ¬richsee,Informatik/Telekommunikation,Projekt Management/Analyse,32.0,2014,IT Projektleiter/-in
2,23,23,jobsse-5673676,2014-02-28,2014-11-03,IT- und Organsations-Projektleiter,<p>Wirtschaftsinformatiker oder Organisator mi...,Informatik/Telekommunikation,Personalberatung,Region Bern,Region Bern,Informatik/Telekommunikation,Projekt Management/Analyse,11.0,2014,IT- und Organsations-Projektleiter
3,34,34,jobsse-6313354,2015-03-17,2015-03-30,IT Gesamtprojektleiter - Senior IT Project Man...,"<p>FÇ¬r unseren Kunden, eines der fÇ¬hrenden U...",Informatik/Telekommunikation,Personalberatung,Region ZÇ¬rich / Schaffhausen,Stadt ZÇ¬rich / ZÇ¬richsee,Informatik/Telekommunikation,Projekt Management/Analyse,13.0,2015,IT Gesamtprojektleiter - Senior IT Project Man...
4,35,35,jobsse-5585493,2014-01-16,2014-12-02,"IT Projektmanager ¶®Application¶¯ (DEUTSCH, FR...",<b>Ihre Herausforderung</b><ul><li>Verantwortl...,Industrie diverse,Personalberatung,Region ZÇ¬rich / Schaffhausen,Region ZÇ¬rich / Schaffhausen,Informatik/Telekommunikation,Netzwerk Spezialisten/Engineers,27.0,2014,"IT Projektmanager ¶®Application¶¯ (DEUTSCH, FR..."


In [23]:
# Lemmatisierung der Titel
start = time()
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

for mail in data['Title_Anforderungen']:
    lemma = [lemma for (word,lemma,pos) in tagger.tag_sent(mail.split())]
    index = data[data['Title_Anforderungen']==mail].index
    data.loc[index, 'Title_Anforderungen'] = (' '.join(lemma))
    
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 216.8528733253479 Sekunden


In [24]:
# Text in Kleinbuchstaben umwandeln
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.lower()

In [25]:
# Satzzeichen löschen
def remove_punctuation(txt):
    txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct

data['Title_Anforderungen'] = data['Title_Anforderungen'].apply(lambda x: remove_punctuation(x))

In [26]:
# Tabs \t aus den Texten entfernen
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("\t", "", regex=True)
data['Title_Anforderungen']

0               projektleiter sap inkasso  exkasso fscd wm
1                                       it projektleiterin
2                         it und organsationsprojektleiter
3        it gesamtprojektleiter  senior it project mana...
4        it projektmanager ¶®application¶¯ deutsch fran...
                               ...                        
25095      sachbearbeiterin buchhaltung per sofort tryhire
25096    sachbearbeiterin administration und buchhaltun...
25097                              jung leiter buchhaltung
25098    sachbearbeiterin buchhaltung  projektunterstüt...
25099    sachbearbeiter buchhaltung als buchhaltungsmit...
Name: Title_Anforderungen, Length: 25100, dtype: object

In [27]:
# Ähnliche Anforderungen manuell mit einem Hauptbegriff ersetzen
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("teamplayer", "teamfähig", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("verhandlungssicher", "verhandlungsgeschick", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("kundenorientierte", "kundenorientierung", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("projekt management", "projektleitungserfahrung", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("projektmanagement", "projektleitungserfahrung", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("zusammenarbeit", "teamfähig", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("kommunikation", "kommunikationsfähigkeit", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("kommunikationsfähigkeiten", "kommunikationsfähigkeit", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("dwh", "data warehouse", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("dienstleistungsorientierung", "dienstleistungsorientiert", regex=True)
data['Title_Anforderungen'] = data['Title_Anforderungen'].str.replace("eigenverantwortung", "eigenverantwortlich", regex=True)

In [28]:
# Word-Tokenization 
#nltk.download('punkt')

start = time()
data['title_tok'] = data.apply(lambda row: nltk.word_tokenize(row['Title_Anforderungen']), axis=1)
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 3.345613479614258 Sekunden


In [29]:
# Stopwörter löschen
nltk.download('stopwords')

stop_words = set(stopwords.words('german'))
data['title_tok'] = data['title_tok'].apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to C:\Users\Sven
[nltk_data]     Konermann\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
# Tokenizing rückgängig machen, da die Tokenizierung nicht in eine csv-Datei überführt werden kann
column = []
for i in data['title_tok']:
    test = TreebankWordDetokenizer().detokenize(i)
    column.append(test)

data['Title_Anforderungen'] = column

In [31]:
# Löschen der tokenisierten Spalte
data = data.drop(['title_tok'], axis = 1)

In [32]:
# Erneutes Löschen der Duplikate, da sich die Texte verändert haben
data = data.drop_duplicates(subset='Title_Anforderungen', keep="last")

In [34]:
data.to_csv("Inserate_clean_Titelanalyse_AF_E.csv", sep=";")